In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

%load_ext tensorboard

In [2]:
# 1. get mnist from tensorflow_datasets
mnist = tfds.load("mnist", split =["train","test"], as_supervised=True)
train_ds = mnist[0]
val_ds = mnist[1]

# 2. write function to create the dataset that we want
def preprocess(data, batch_size, target_function):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/128.)-1., t))
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    #for subtask a: map ((x1,y1),(x2,y2)) to (x1,x2, y1+y2 >= 5*) *boolean
    #for loss use binary cross entropy and for activation function sigmoid
    #for subtask b: output layer is vector for values betweeen -9 and +9 -> turn into one_hot_vector
    #for loss use categorical cross entropy and for activation function softmax
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], target_function(x1[1], x2[1])))
    if type(zipped_ds.t) == bool:
        # transform boolean target to int
        zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))
    else:
        #create one-hot targets
        mnist = mnist.map(lambda img, target: (img, tf.one_hot(target, depth=19)))
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds

train_ds = preprocess(train_ds, batch_size=32) #train_ds.apply(preprocess)
val_ds = preprocess(val_ds, batch_size=32) #val_ds.apply(preprocess)



2022-11-20 12:44:47.249134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 12:44:47.250908: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
# check the contents of the dataset
for img1, img2, label in train_ds.take(1):
    print(img1.shape, img2.shape, label.shape)

(32, 784) (32, 784) (32,)


2022-11-20 12:46:36.485314: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-11-20 12:46:36.488758: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:


class TwinMNISTModel(tf.keras.Model):

    # 1. constructor
    def __init__(self, hidden_activation, output_activation):
        super().__init__()
        # inherit functionality from parent class

        # optimizer, loss function and metrics
        self.metrics_list = [tf.keras.metrics.BinaryAccuracy(),
                             tf.keras.metrics.Mean(name="loss")]
        
        self.optimizer = tf.keras.optimizers.Adam()
        
        self.loss_function = tf.keras.losses.BinaryCrossentropy()
        
        # layers to be used
        self.dense1 = tf.keras.layers.Dense(32, activation=hidden_activation) #a: ? b: ?
        self.dense2 = tf.keras.layers.Dense(32, activation=hidden_activation)
        
        self.out_layer = tf.keras.layer.Dense(1,activation=output_activation) #a: sigmoid b: softmax
        
        
        
        
    # 2. call method (forward computation)
    def call(self, images, training=False):
        img1, img2 = images
        
        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)
        
        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)
        
        combined_x = tf.concat([img1_x, img2_x ], axis=1)
        
        return self.out_layer(combined_x)



    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model



    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metrics.reset_states()



    # 5. train step method
    def train_step(self, data):
        img1, img2, label = data
        
        with tf.GradientTape() as tape:
            output = self((img1, img2), training=True)
            loss = self.loss_function(label, output)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        
        # update the state of the metrics according to loss
        # return a dictionary with metric names as keys and metric results as values



    # 6. test_step method
    def test(model, test_data, loss_function):
        # test over complete test data

        #record accuracy and loss in list
        test_accuracy_aggregator = []
        test_loss_aggregator = []

        for (input, target) in test_data:
            #perform forward step
            prediction = model(input)
            #calculate loss and accuracy
            sample_test_loss = loss_function(target, prediction) #a: binary cross entropy b: categorical cross entropy
            sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
            sample_test_accuracy = np.mean(sample_test_accuracy)
            #append accuracy and loss to list
            test_loss_aggregator.append(sample_test_loss.numpy())
            test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

        #calculate mean of loss and accuracy to return
        test_loss = tf.reduce_mean(test_loss_aggregator)
        test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

        return test_loss, test_accuracy

